<a href="https://colab.research.google.com/github/HannahCarnes/Home_sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [ ]:
# 2. Create a temporary view of the DataFrame.
df = spark.read.csv(url, header=True, inferSchema=True)
df.createOrReplaceTempView("home_sales_view")


In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
four_bedroom_query = """
SELECT year, ROUND(AVG(price), 2) AS average_price
FROM home_sales_view
WHERE bedrooms = 4
GROUP BY year
ORDER BY year
"""

result = spark.sql(four_bedroom_query)

# Show result
result.show()


In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

Three_bedroom_query = """
SELECT year_built, ROUND(AVG(price), 2) AS average_price
FROM home_sales_view
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year_built
ORDER BY year_built
"""

result = spark.sql(3_bedroom_query)

# show result
result.show()


In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
three_bed_bath_query = """
SELECT year_built, ROUND(AVG(price), 2) AS average_price
FROM home_sales_view
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft >= 2000
GROUP BY year_built
ORDER BY year_built
"""

result = spark.sql(three_bed_bath_query)

# Show result
result.show()


In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

view_query = """
SELECT AVG(price) AS average_price
FROM home_sales_view
WHERE price >= 350000
"""

result = spark.sql(view_query)
average_price = result.collect()[0]["average_price"]
view_rating = round(average_price / 10000, 2)

print("--- %s seconds ---" % (time.time() - start_time))

--- 7.200241088867188e-05 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.table("home_sales").cache()

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
query_cached = """
SELECT AVG(price) AS average_price
FROM home_sales_view
WHERE price >= 350000
"""


print("--- %s seconds ---" % (time.time() - start_time))


--- 5.459785461425781e-05 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
partitioned_parquet_path = "/content/partitioned_home_sales.parquet"
df.write.partitionBy("date_built").parquet(partitioned_parquet_path)

In [ ]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet(partitioned_parquet_path)

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("partitioned_home_sales_view")

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

parquet_view_query = """
SELECT AVG(price) AS average_price
FROM partitioned_home_sales_view
WHERE price >= 350000
"""

parquet_result = spark.sql(parquet_view_query)

print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales_view")

In [ ]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached('home_sales_view')
